In [6]:
import pandas as pd
import requests
import sys
import time

noaa_key = "JOwIFUXjlWBqtstSqsSaKYDMzShAccQe"
usda_key = "8D5EDCA6-733D-3ACA-AF06-BA5BB3938B4F"

In [31]:
import requests

def get_usda_data(usda_key, year):
  # API endpoint
  url = "https://quickstats.nass.usda.gov/api/api_GET/"

  params = {
    "key": usda_key,
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "unit_desc": "BU / ACRE",
    "state_alpha": "MI",
    "year": year,
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "agg_level_desc": "COUNTY",
    "format": "JSON"
  }

  response = requests.get(url, params=params)

  if response.status_code == 200:
    data = response.json()
    return pd.DataFrame(data['data'])
  else:
    print(f"Request failed with status code {response.status_code}")
    return None

In [8]:
pd.set_option('display.max_rows', 10) 
usda_df = pd.DataFrame(data['data'])
usda_df

,Value,watershed_desc,country_code,statisticcat_desc,util_practice_desc,agg_level_desc,county_code,prodn_practice_desc,state_ansi,short_desc,...,region_desc,unit_desc,state_name,source_desc,reference_period_desc,sector_desc,congr_district_code,asd_desc,begin_code,class_desc
0,89.7,,9000,YIELD,GRAIN,COUNTY,041,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,UPPER PENINSULA,00,ALL CLASSES
1,108.9,,9000,YIELD,GRAIN,COUNTY,043,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,UPPER PENINSULA,00,ALL CLASSES
2,91.3,,9000,YIELD,GRAIN,COUNTY,109,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,UPPER PENINSULA,00,ALL CLASSES
3,112.2,,9000,YIELD,GRAIN,COUNTY,029,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,NORTHWEST,00,ALL CLASSES
4,106,,9000,YIELD,GRAIN,COUNTY,047,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,NORTHWEST,00,ALL CLASSES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,142.6,,9000,YIELD,GRAIN,COUNTY,125,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,SOUTHEAST,00,ALL CLASSES
56,146.7,,9000,YIELD,GRAIN,COUNTY,147,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,SOUTHEAST,00,ALL CLASSES
57,165.8,,9000,YIELD,GRAIN,COUNTY,161,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,SOUTHEAST,00,ALL CLASSES
58,120.4,,9000,YIELD,GRAIN,COUNTY,163,ALL PRODUCTION PRACTICES,26,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",...,,BU / ACRE,MICHIGAN,SURVEY,YEAR,CROPS,,SOUTHEAST,00,ALL CLASSES


In [ ]:
def is_valid_data(data):
  return data and isinstance(data, dict) and "results" in data and data["results"]

# Sate in USDA data is state_ansi
def get_raw_weather(state_ansi, county_ansi, startdate, enddate):
  url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

  headers = {
  'token': noaa_key
  }

  # Convert startdate and enddate to datetime objects
  startdate = pd.to_datetime(startdate)
  enddate = pd.to_datetime(enddate)

  # Calculate the number of days between startdate and enddate
  num_days = (enddate - startdate).days

  # Check if the number of days is greater than 30
  if num_days > 30:
    # Split the date range into multiple requests
    requests_list = []
    current_date = startdate
    while current_date <= enddate:
      # Calculate the end date for the current request
      current_enddate = current_date + pd.DateOffset(days=30)
      if current_enddate > enddate:
        current_enddate = enddate

      # Create the parameters for the current request
      params = {
        "datasetid": "GHCND",
        "startdate": current_date.strftime("%Y-%m-%d"),
        "enddate": current_enddate.strftime("%Y-%m-%d"),
        "units": "standard",
        "limit": "1000", # 1000 is the max limit
        "locationid": f"FIPS:{state_ansi}{county_ansi}",
        'datatypeid': ['TMAX', 'TMIN']
      }

      # Make the request
      print(f"FIPS:{state_ansi}{county_ansi}")
      response = requests.get(url, headers=headers, params=params)

      if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
          # Append the results to the list
          requests_list.append(pd.DataFrame(data["results"]))
        else:
          print(f"No data found for FIPS:{params['locationid']}")
      elif response.status_code == 503:
        print("Server error. Retrying in 3 seconds...")
        time.sleep(3)
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
          data = response.json()
          if is_valid_data(data):
            # Append the results to the list
            requests_list.append(pd.DataFrame(data["results"]))
          else:
            print(f"No data found for FIPS:{params['locationid']} after retry")
        else:
          print(response.text)
      else:
        print(response.text)

      # Move to the next date range
      current_date = current_enddate + pd.DateOffset(days=1)

    # Concatenate the results from all requests
    if requests_list:
      return pd.concat(requests_list)
    else:
      return None
  else:
    # Single request for the entire date range
    params = {
      "datasetid": "GHCND",
      "startdate": startdate.strftime("%Y-%m-%d"),
      "enddate": enddate.strftime("%Y-%m-%d"),
      "units": "standard",
      "limit": "1000",
      "locationid": f"FIPS:{state_ansi}{county_ansi}",
      'datatypeid': ['TMAX', 'TMIN']
    }
    print(f"FIPS:{state_ansi}{county_ansi}")
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
      data = response.json()
      # Return data as a DataFrame
      print(data)
      # Check if 'results' key is present in the response
      if is_valid_data(data):
        return pd.DataFrame(data["results"])
      else:
        print(f"No data found for FIPS:{params['locationid']}")
        return None
    else:
      print(response.text)
      return None

In [10]:
weather_raw = get_raw_weather("26", "021", "2022-12-20T00:00:00", "2022-12-31T00:00:00")

# Save the raw weather data to a CSV file
weather_raw.to_csv("weather_raw.csv", index=False)

FIPS:26021
{'metadata': {'resultset': {'offset': 1, 'count': 72, 'limit': 1000}}, 'results': [{'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 29.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 21.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00205892', 'attributes': ',,7,1500', 'value': 35.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00205892', 'attributes': ',,7,1500', 'value': 23.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USW00094871', 'attributes': ',,W,2400', 'value': 34.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USW00094871', 'attributes': ',,W,2400', 'value': 22.0}, {'date': '2022-12-21T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 32.0}, {'date': '2022-12-21T00:

In [43]:
def clean_weather(df,state_ansi,county_ansi):
    # For each day in date column average the TMAX and TMIN into their own columns across all stations
    df["date"] = pd.to_datetime(df["date"])
    df["TMAX"] = df["value"].where(df["datatype"] == "TMAX")
    df["TMIN"] = df["value"].where(df["datatype"] == "TMIN")
    
    df = df.groupby("date").agg({"TMAX": "mean", "TMIN": "mean"}).reset_index()

    # add state and county columns
    df["state_ansi"] = state_ansi
    df["county_ansi"] = county_ansi
    
    return df

In [12]:
weather = clean_weather(weather_raw, "26", "145")

# Print the first 25 rows of the weather DataFrame
weather.head(25)
weather.to_csv("weather.csv", index=False)

In [16]:
# for each state_ansi county_ansi combo in the USDA data, get the weather data

def get_weather_data(usda_df):
    weather_data = []
    for index, row in usda_df.iterrows():
        state_ansi = row["state_ansi"]
        county_ansi = row["county_ansi"]
        #time.sleep(1)
        weather_raw = get_raw_weather(state_ansi, county_ansi, "2022-10-20T00:00:00", "2022-12-31T00:00:00")
        if weather_raw is not None:
            weather = clean_weather(weather_raw, state_ansi, county_ansi)
            weather_data.append(weather)
    return pd.concat(weather_data)

weather_data = get_weather_data(usda_df)
weather_data.to_csv("weather_data.csv", index=False)

FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26043
FIPS:26043
FIPS:26043
FIPS:26109
Server error. Retrying in 5 seconds...
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26029
FIPS:26029
FIPS:26029
FIPS:26047
FIPS:26047
FIPS:26047
FIPS:26055
FIPS:26055
FIPS:26055
FIPS:26089
FIPS:26089
FIPS:26089
FIPS:26101
FIPS:26101
FIPS:26101
FIPS:26113
FIPS:26113
FIPS:26113
FIPS:26165
FIPS:26165
FIPS:26165
FIPS:26007
FIPS:26007
FIPS:26007
FIPS:26069
FIPS:26069
FIPS:26069
FIPS:26129
FIPS:26129
FIPS:26129
FIPS:26137
FIPS:26137
FIPS:26137
FIPS:26141
FIPS:26141
FIPS:26141
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26121
FIPS:26121
FIPS:26121
FIPS:26123
FIPS:26123
FIPS:26123
FIPS:26127
FIPS:26127
FIPS:26127
FIPS:26035
No data found for FIPS:FIPS:26035
FIPS:26035
No data found for FIPS:FIPS:26035
FIPS:26035
No data found for FIPS:FIPS:2

In [14]:
# Count the number of unique items in column county for weather_data
weather_data["county"].nunique()

56

In [44]:
# Define a list of 50 years from 1970 to 2019
# Get USDA data for each year
# Get get_weather_data for each year
# Save the data to a CSV file

years = range(2019, 2020)

usda_data = []
weather_data = []

for year in years:
  print(year)
  usda_data.append(pd.DataFrame(get_usda_data(usda_key, year)))
  weather_data.append(pd.DataFrame(get_weather_data(usda_df)))

usda_data_df = pd.concat(usda_data)
weather_data_df = pd.concat(weather_data)

usda_data_df.to_csv("usda_data.csv", index=False)
weather_data_df.to_csv("weather_data.csv", index=False)

2019
FIPS:26041
FIPS:26041
Server error. Retrying in 5 seconds...
FIPS:26041
FIPS:26043
FIPS:26043
FIPS:26043
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26029
FIPS:26029
FIPS:26029
FIPS:26047
FIPS:26047
FIPS:26047
FIPS:26055
FIPS:26055
FIPS:26055
FIPS:26089
FIPS:26089
FIPS:26089
FIPS:26101
FIPS:26101
FIPS:26101
FIPS:26113
FIPS:26113
FIPS:26113
FIPS:26165
FIPS:26165
FIPS:26165
FIPS:26007
FIPS:26007
FIPS:26007
FIPS:26069
FIPS:26069
FIPS:26069
FIPS:26129
FIPS:26129
FIPS:26129
FIPS:26137
FIPS:26137
FIPS:26137
FIPS:26141
FIPS:26141
FIPS:26141
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26105
No data found for FIPS:FIPS:26105
FIPS:26121
FIPS:26121
FIPS:26121
FIPS:26123
FIPS:26123
FIPS:26123
FIPS:26127
FIPS:26127
FIPS:26127
FIPS:26035
No data found for FIPS:FIPS:26035
FIPS:26035
No data found for FIPS:FIPS:26035
FIPS:26035
No data found for FIPS:F

In [45]:
usda_data_df

,country_code,commodity_desc,state_fips_code,sector_desc,end_code,region_desc,CV (%),state_ansi,prodn_practice_desc,Value,...,statisticcat_desc,country_name,begin_code,domaincat_desc,agg_level_desc,group_desc,asd_code,week_ending,county_code,year
0,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,117.9,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,10,,998,2019
1,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,116.7,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,10,,041,2019
2,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,73.2,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,10,,109,2019
3,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,116.1,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,20,,998,2019
4,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,91.7,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,20,,019,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,178.7,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,90,,115,2019
52,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,133.3,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,90,,125,2019
53,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,146.3,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,90,,147,2019
54,9000,CORN,26,CROPS,00,,,26,ALL PRODUCTION PRACTICES,154.2,...,YIELD,UNITED STATES,00,NOT SPECIFIED,COUNTY,FIELD CROPS,90,,161,2019


In [46]:
weather_data_df

,date,TMAX,TMIN,state_ansi,county_ansi
0,2022-10-20,46.500000,32.500000,26,041
1,2022-10-21,57.500000,36.000000,26,041
2,2022-10-22,58.500000,31.500000,26,041
3,2022-10-23,60.500000,42.500000,26,041
4,2022-10-24,65.000000,53.000000,26,041
...,...,...,...,...,...
6,2022-11-22,38.432099,22.432099,26,
7,2022-12-21,26.440476,10.559524,26,
8,2022-12-22,29.420732,12.250000,26,
9,2022-12-23,29.895062,8.438272,26,
